In [ ]:
from math import isclose
ops = {
    '+': (2, 'left'),
    '-': (2, 'left'),
    '*': (3, 'left'),
    '/': (3, 'left'),
    '^': (4, 'right'),
}

In [ ]:
def tokenize(expr: str):
    tokens = []
    i, prev = 0, None
    while i < len(expr):
        c = expr[i]
        if c.isspace():
            i += 1; continue
        if c.isdigit() or c == '.':
            num = [c]; i += 1
            while i < len(expr) and (expr[i].isdigit() or expr[i] == '.'):
                num.append(expr[i]); i += 1
            tokens.append(float(''.join(num)) if '.' in num else int(''.join(num)))
            prev = 'num'; continue
        if c in '+-':
            if prev is None or prev in ('op', 'lp'):
                if c == '+': i += 1; prev = 'op'; continue
                else: tokens += [0, '-']; i += 1; prev = 'op'; continue
            else: tokens.append(c); prev = 'op'; i += 1; continue
        if c in '*/^': tokens.append(c); prev = 'op'; i += 1; continue
        if c == '(' : tokens.append(c); prev = 'lp'; i += 1; continue
        if c == ')' : tokens.append(c); prev = 'rp'; i += 1; continue
        raise ValueError(f"Bad char: {c}")
    return tokens

In [ ]:
def shunting_yard(tokens):
    out, st = [], []
    for tok in tokens:
        if isinstance(tok,(int,float)): out.append(tok)
        elif tok in ops:
            p,a = ops[tok]
            while st and st[-1] in ops:
                tp,_ = ops[st[-1]]
                if (a=='left' and p<=tp) or (a=='right' and p<tp): out.append(st.pop())
                else: break
            st.append(tok)
        elif tok == '(' : st.append(tok)
        elif tok == ')' :
            while st and st[-1] != '(': out.append(st.pop())
            if not st: raise ValueError("Mismatched )")
            st.pop()
    while st:
        if st[-1] in '()': raise ValueError("Mismatched ()")
        out.append(st.pop())
    return out

In [ ]:
def eval_rpn(rpn):
    st = []
    for tok in rpn:
        if isinstance(tok,(int,float)): st.append(float(tok))
        else:
            b,a = st.pop(), st.pop()
            if tok=='+': st.append(a+b)
            elif tok=='-': st.append(a-b)
            elif tok=='*': st.append(a*b)
            elif tok=='/':
                if isclose(b,0): raise ZeroDivisionError("Division by zero")
                st.append(a/b)
            elif tok=='^': st.append(a**b)
    res = st[0]
    return int(round(res)) if isclose(res,round(res)) else res

In [ ]:
def evaluate(expr: str): return eval_rpn(shunting_yard(tokenize(expr)))

In [ ]:
if __name__ == "__main__":
    print("Введи вираз (exit для виходу):")
    while True:
        s = input("> ").strip()
        if s.lower() in ("exit","quit"): break
        try: print(evaluate(s))
        except Exception as e: print("Помилка:", e)